In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

import numpy as np
import io

from PIL import Image

import pandas as pd

In [ ]:
# split = "test"


split = "train"
# split = "val"

split_folder = Path(f"/media/nvme1/argoverse/sensor_mini/{split}")
log_folder = sorted(list(split_folder.iterdir()))[2]

log_folder.name

In [ ]:
import json
from typing import Dict, List

from d123.common.geometry.line.polylines import Polyline3D
from d123.dataset.dataset_specific.av2.av2_map_conversion import _extract_lane_group_dict


def _extract_polyline(data: List[Dict[str, float]], close: bool = False) -> Polyline3D:
    polyline = np.array([[p["x"], p["y"], p["z"]] for p in data], dtype=np.float64)
    if close:
        polyline = np.vstack([polyline, polyline[0]])

    return Polyline3D.from_array(polyline)


map_folder = log_folder / "map"
log_map_archive_path = next(map_folder.glob("log_map_archive_*.json"))

with open(log_map_archive_path, "r") as f:
    log_map_archive = json.load(f)

drivable_areas: Dict[int, Polyline3D] = {}

for drivable_area_id, drivable_area_dict in log_map_archive["drivable_areas"].items():
    # keys: ["area_boundary", "id"]
    drivable_areas[int(drivable_area_id)] = _extract_polyline(drivable_area_dict["area_boundary"], close=True)

for lane_segment_id, lane_segment_dict in log_map_archive["lane_segments"].items():
    # keys = [
    #     "id",
    #     "is_intersection",
    #     "lane_type",
    #     "left_lane_boundary",
    #     "left_lane_mark_type",
    #     "right_lane_boundary",
    #     "right_lane_mark_type",
    #     "successors",
    #     "predecessors",
    #     "right_neighbor_id",
    #     "left_neighbor_id",
    # ]
    lane_segment_dict["left_lane_boundary"] = _extract_polyline(lane_segment_dict["left_lane_boundary"])
    lane_segment_dict["right_lane_boundary"] = _extract_polyline(lane_segment_dict["right_lane_boundary"])

for crosswalk_id, crosswalk_dict in log_map_archive["pedestrian_crossings"].items():
    # keys = ["id", "outline"]
    # https://github.com/argoverse/av2-api/blob/6b22766247eda941cb1953d6a58e8d5631c561da/src/av2/map/pedestrian_crossing.py

    p1, p2 = np.array([[p["x"], p["y"], p["z"]] for p in crosswalk_dict["edge1"]], dtype=np.float64)
    p3, p4 = np.array([[p["x"], p["y"], p["z"]] for p in crosswalk_dict["edge2"]], dtype=np.float64)
    crosswalk_dict["outline"] = Polyline3D.from_array(np.array([p1, p2, p4, p3, p1], dtype=np.float64))


lane_group_dict = _extract_lane_group_dict(log_map_archive["lane_segments"])

In [ ]:
from typing import Any

import shapely

from d123.common.geometry.base import Point3DIndex
import geopandas as gpd

from d123.common.geometry.line.polylines import Polyline2D
from d123.common.geometry.occupancy_map import OccupancyMap2D

import numpy.typing as npt


def _interpolate_z_on_segment(point: shapely.Point, segment_coords: npt.NDArray[np.float64]) -> float:
    """Interpolate Z coordinate along a 3D line segment."""
    p1, p2 = segment_coords[0], segment_coords[1]

    # Project point onto segment
    segment_vec = p2[:2] - p1[:2]
    point_vec = np.array([point.x, point.y]) - p1[:2]

    # Handle degenerate case
    segment_length_sq = np.dot(segment_vec, segment_vec)
    if segment_length_sq == 0:
        return p1[2]

    # Calculate projection parameter
    t = np.dot(point_vec, segment_vec) / segment_length_sq
    t = np.clip(t, 0, 1)  # Clamp to segment bounds

    # Interpolate Z
    return p1[2] + t * (p2[2] - p1[2])


def _extract_intersection_dict(
    lanes: Dict[int, Any], lane_group_dict: Dict[int, Any], max_distance: float = 0.01
) -> Dict[str, Any]:

    # 1. Collect all lane groups where at least one lane is marked as an intersection.
    lane_group_intersection_dict = {}
    for lane_group_id, lane_group in lane_group_dict.items():
        is_intersection_lanes = [lanes[str(lane_id)]["is_intersection"] for lane_id in lane_group["lane_ids"]]
        if any(is_intersection_lanes):
            lane_group_intersection_dict[lane_group_id] = lane_group

    # 2. Merge polygons of lane groups that are marked as intersections.
    lane_group_intersection_geometry = {
        lane_group_id: shapely.Polygon(lane_group["outline"].array[:, Point3DIndex.XY])
        for lane_group_id, lane_group in lane_group_intersection_dict.items()
    }
    intersection_polygons = gpd.GeoSeries(lane_group_intersection_geometry).union_all()

    # 3. Collect all intersection polygons and their lane group IDs.
    intersection_dict = {}
    for intersection_idx, intersection_polygon in enumerate(intersection_polygons.geoms):
        if intersection_polygon.is_empty:
            continue
        lane_group_ids = [
            lane_group_id
            for lane_group_id, lane_group_polygon in lane_group_intersection_geometry.items()
            if intersection_polygon.intersects(lane_group_polygon)
        ]
        intersection_dict[f"intersection_{intersection_idx}"] = {
            "id": intersection_idx,
            "outline_2d": Polyline2D.from_array(np.array(list(intersection_polygon.exterior.coords), dtype=np.float64)),
            "lane_group_ids": lane_group_ids,
        }

    # 4. Lift intersection outlines to 3D.
    boundary_segments = []
    for lane_group in lane_group_intersection_dict.values():
        coords = np.array(lane_group["outline"].linestring.coords, dtype=np.float64).reshape(-1, 1, 3)
        segment_coords_boundary = np.concatenate([coords[:-1], coords[1:]], axis=1)
        boundary_segments.append(segment_coords_boundary)

    boundary_segments = np.concatenate(boundary_segments, axis=0)
    boundary_segment_linestrings = shapely.creation.linestrings(boundary_segments)
    occupancy_map = OccupancyMap2D(boundary_segment_linestrings)

    for intersection_id, intersection_data in intersection_dict.items():
        points_2d = intersection_data["outline_2d"].array
        points_3d = np.zeros((len(points_2d), 3), dtype=np.float64)
        points_3d[:, :2] = points_2d

        query_points = shapely.creation.points(points_2d)
        results = occupancy_map.query_nearest(query_points, max_distance=max_distance, exclusive=True)
        for query_idx, geometry_idx in zip(*results):
            query_point = query_points[query_idx]
            segment_coords = boundary_segments[geometry_idx]
            best_z = _interpolate_z_on_segment(query_point, segment_coords)
            points_3d[query_idx, 2] = best_z

        intersection_dict[intersection_id]["outline_3d"] = Polyline3D.from_array(points_3d)

    return intersection_dict


intersection_dict = _extract_intersection_dict(log_map_archive["lane_segments"], lane_group_dict)

In [ ]:
from matplotlib import cm
from matplotlib.colors import Normalize


fig, ax = plt.subplots(figsize=(10, 10))

for intersection_id, values in intersection_dict.items():

    outline = values["outline_3d"].array
    print(outline[:, 2].min(), outline[:, 2].max())

    # Normalize z values to [0, 1] for colormap mapping
    norm = Normalize(vmin=outline[:, 2].min(), vmax=outline[:, 2].max())
    colors = cm.viridis(norm(outline[:, 2]))

    # Plot each segment with its corresponding color
    for i in range(len(outline) - 1):
        ax.plot(outline[i : i + 2, 0], outline[i : i + 2, 1], color=colors[i], linewidth=2)

ax.set_aspect("equal")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))


for lane in log_map_archive["lane_segments"].values():
    left_boundary = lane["left_lane_boundary"]
    right_boundary = lane["right_lane_boundary"]

    polygon = np.concatenate([left_boundary.array, right_boundary.array[::-1]]).reshape(-1, 3)[:, :2]
    print(polygon)
    ax.fill(
        polygon[:, 0], polygon[:, 1], alpha=0.5, edgecolor="black", color="red" if lane["is_intersection"] else "blue"
    )

    # if left_boundary and right_boundary:
    #     ax.plot(left_boundary.array[:, 0], left_boundary.array[:, 1], color="blue", linewidth=1)
    #     ax.plot(right_boundary.array[:, 0], right_boundary.array[:, 1], color="red", linewidth=1)

ax.set_title("Lane Segments")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_aspect("equal")
plt.show()